# Rx 数据流

RxJs一切都已数据流为中心

与数据流相关操作符（绝大部分为静态操作符） -- 任何一个操作符都会返回Observable对象，创建类操作符不依赖于其他Observable对象

## 同步数据流

包括函数`create`、`of`

In [2]:
import { Observable } from 'rxjs/Observable'
var source$ = Observable.create(observer => {
    observer.next(1);
    observer.next(2);
    observer.next(3);
});
source$.subscribe(console.log);

1
2
3


Subscriber {
  closed: false,
  _parentOrParents: null,
  _subscriptions: null,
  syncErrorValue: null,
  syncErrorThrown: false,
  syncErrorThrowable: true,
  isStopped: false,
  destination: SafeSubscriber {
    closed: false,
    _parentOrParents: null,
    _subscriptions: null,
    syncErrorValue: null,
    syncErrorThrown: false,
    syncErrorThrowable: false,
    isStopped: false,
    destination: {
      closed: true,
      next: [Function: next],
      error: [Function: error],
      complete: [Function: complete]
    },
    _parentSubscriber: [Circular],
    _context: [Circular],
    _next: [Function: bound log],
    _error: undefined,
    _complete: undefined
  }
}

In [1]:
import { Observable } from 'rxjs/Observable'
import 'rxjs/add/observable/of'
// of 操作符
var source$ = Observable.of(1, 2, 3);
var subscrition = source$.subscribe(console.log)

Error: SyntaxError

In [6]:
// import {  } from 'rxjs/Observable';